### Title Generation
BIG KINDS에서 다운로드 받은 뉴스 기사 중 100000개를 선정하여 사용한다.

### 필요 라이브러리 import

In [1]:
import numpy as np
import pandas as pd
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import Embedding, Input, Bidirectional, LSTM, Dense, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import pad_sequences

In [2]:
# CSV 파일 읽기
df = pd.read_csv('Merged_NewsResults.csv')[:8000]
df

,제목,본문
0,음악으로 교감하는 싱글맘과 반항아 아들 영화 '플로라 앤 썬',아일랜드 더블린의 한 동네 클럽에서 열리는 아마추어 동네 음악 경연 대회. 이제 열...
1,오뚜기와 해태 설립의 '나비효과' 김치볶음밥의 탄생[책마을],프라이팬이 한국에 전해진 것은 일제강점기였지만 대중화된 것은 1970년대부터였다. ...
2,'수산물 소비' 식당 시장 희비에도 손님은 '불안불안',지금 당장은 괜찮을 것 같아 먹긴 하지만 불안이 가시진 않는다. 나중이 걱정이죠.일...
3,"수원 국회의원, 7개 공통 공약 중 3년간 3개 이행 [경기 인천 국회의원 공약 점검 ]","사진 왼쪽부터 김승원, 백혜련, 김영진, 박광온, 김진표 국회의원. 의원실 제공 ..."
4,"수원 미완 공약, 22대 국회로 넘어가나 [경기 인천 국회의원 공약 점검 ]",수원군공항 모습. 경기일보DB 수원특례시 국회의원들이 제시한 7대 공통 공약 중 ...
...,...,...
7995,죽 쒔던 퇴직연금 증시 반등에 화색,올해 2분기 원리금비보장 퇴직연금 손익률이 일제히 수익으로 전환했다. 두 자릿수 손...
7996,이수과천 복합터널 건설 본궤도 2030년 개통,동작대로의 상습 정체를 해결하기 위한 '이수과천 복합터널' 건설사업이 본궤도에 올랐...
7997,[속보] 민주당 40대 김포시의원 길가서 숨진 채 발견 부검 의뢰 예정,더불어민주당 소속 김포시의원이 길가에서 숨진 채 발견돼 경찰이 수사에 나섰다.20일...
7998,"신세계그룹, CEO 40 교체 쇄신인사 이마트 유통 3사 한채양 원톱 체제로",신세계그룹이 변화와 쇄신을 키워드로 2024년 정기 임원인사를 단행했다. 이번 인사...


### 간단한 데이터 전처리
한글과 알파벳을 제외한 문자들을 제거한다.

In [3]:
df['본문'] = df['본문'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ]", "", regex=True)
df['제목'] = df['제목'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ]", "", regex=True)

In [4]:
df

,제목,본문
0,음악으로 교감하는 싱글맘과 반항아 아들 영화 플로라 앤 썬,아일랜드 더블린의 한 동네 클럽에서 열리는 아마추어 동네 음악 경연 대회 이제 열네...
1,오뚜기와 해태 설립의 나비효과 김치볶음밥의 탄생책마을,프라이팬이 한국에 전해진 것은 일제강점기였지만 대중화된 것은 년대부터였다 오뚜기와 ...
2,수산물 소비 식당 시장 희비에도 손님은 불안불안,지금 당장은 괜찮을 것 같아 먹긴 하지만 불안이 가시진 않는다 나중이 걱정이죠일본 ...
3,수원 국회의원 개 공통 공약 중 년간 개 이행 경기 인천 국회의원 공약 점검,사진 왼쪽부터 김승원 백혜련 김영진 박광온 김진표 국회의원 의원실 제공 대 국회의...
4,수원 미완 공약 대 국회로 넘어가나 경기 인천 국회의원 공약 점검,수원군공항 모습 경기일보DB 수원특례시 국회의원들이 제시한 대 공통 공약 중 대 ...
...,...,...
7995,죽 쒔던 퇴직연금 증시 반등에 화색,올해 분기 원리금비보장 퇴직연금 손익률이 일제히 수익으로 전환했다 두 자릿수 손실 ...
7996,이수과천 복합터널 건설 본궤도 년 개통,동작대로의 상습 정체를 해결하기 위한 이수과천 복합터널 건설사업이 본궤도에 올랐다 ...
7997,속보 민주당 대 김포시의원 길가서 숨진 채 발견 부검 의뢰 예정,더불어민주당 소속 김포시의원이 길가에서 숨진 채 발견돼 경찰이 수사에 나섰다일 경기...
7998,신세계그룹 CEO 교체 쇄신인사 이마트 유통 사 한채양 원톱 체제로,신세계그룹이 변화와 쇄신을 키워드로 년 정기 임원인사를 단행했다 이번 인사는 성과총...


### Dataset Split
Encoder용 데이터와 Decoder용 데이터를 분할한다.

In [5]:
# 입력 데이터와 타겟 데이터 생성
encoder_texts = df['제목'][:4000].astype(str).to_list()
decoder_texts = df['제목'][4000:8000].astype(str).to_list()

In [6]:
len(encoder_texts), len(decoder_texts)

(4000, 4000)

In [7]:
# 데이터 분리
encoder_texts_train, encoder_texts_test, decoder_texts_train, decoder_texts_test = train_test_split(
    encoder_texts, 
    decoder_texts, 
    test_size=0.2, 
    random_state=42
)

In [8]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [9]:
mecab = Mecab(r"C:\mecab\mecab-ko-dic")

encoder_train_texts_tokenized = [mecab.morphs(item) for item in encoder_texts_train if item not in stopwords]
encoder_test_texts_tokenized = [mecab.morphs(item) for item in encoder_texts_test if item not in stopwords]
decoder_train_texts_tokenized = [mecab.morphs(item) for item in decoder_texts_train if item not in stopwords]
decoder_test_texts_tokenized = [mecab.morphs(item) for item in decoder_texts_test if item not in stopwords]

In [10]:
encoder_train_texts = []
encoder_test_texts = []
decoder_train_texts = []
decoder_test_texts = []

for row in encoder_train_texts_tokenized:
    for col in row:
        encoder_train_texts.append(col)
        
for row in encoder_test_texts_tokenized:
    for col in row:
        encoder_test_texts.append(col)
        
for row in decoder_train_texts_tokenized:
    for col in row:
        decoder_train_texts.append(col)
        
for row in decoder_test_texts_tokenized:
    for col in row:
        decoder_test_texts.append(col)

### 토큰화
keras에서 제공하는 Tokenizer의 text_to_sequences로 토큰화한다.

In [11]:
# 토큰화
tokenizer_encoder = Tokenizer()
tokenizer_encoder.fit_on_texts(encoder_texts_train)
num_encoder_tokens = len(tokenizer_encoder.word_index) + 1

tokenizer_decoder = Tokenizer()
tokenizer_decoder.fit_on_texts(decoder_texts_train)
num_decoder_tokens = len(tokenizer_decoder.word_index) + 1

In [12]:
encoder_sequences_train = tokenizer_encoder.texts_to_sequences(encoder_texts_train)
encoder_sequences_test = tokenizer_encoder.texts_to_sequences(encoder_texts_test)

decoder_sequences_train = tokenizer_decoder.texts_to_sequences(decoder_texts_train)
decoder_sequences_test = tokenizer_decoder.texts_to_sequences(decoder_texts_test)

### 패딩을 진행한다.

In [13]:
max_encoder_seq_length = max(len(seq) for seq in encoder_sequences_train)
max_decoder_seq_length = max(len(seq) for seq in decoder_sequences_train)

encoder_input_data_train = pad_sequences(encoder_sequences_train, maxlen=max_encoder_seq_length, padding='post')
encoder_input_data_test = pad_sequences(encoder_sequences_test, maxlen=max_encoder_seq_length, padding='post')

decoder_input_data_train = pad_sequences(decoder_sequences_train, maxlen=max_decoder_seq_length, padding='post')
decoder_input_data_test = pad_sequences(decoder_sequences_test, maxlen=max_decoder_seq_length, padding='post')

In [14]:
# decoder ont-hot encoding
decoder_output_data_train = np.zeros((len(decoder_sequences_train), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_output_data_test = np.zeros((len(decoder_sequences_test), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, seqs in enumerate(decoder_sequences_train):
    for j, seq in enumerate(seqs):
        decoder_output_data_train[i, j, seq] = 1.

for i, seqs in enumerate(decoder_sequences_test):
    for j, seq in enumerate(seqs):
        decoder_output_data_test[i, j, seq] = 1.

In [15]:
# 모델 정의
latent_dim = 100  # 잠재 공간의 차원

# 인코더
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, latent_dim, mask_zero=True)(encoder_inputs)
encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True))
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_embedding)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

In [16]:
# 디코더
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_decoder_tokens, latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [17]:
# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 컴파일
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
checkpoint = ModelCheckpoint('seq2seq_model.h5', save_best_only=True)

model.fit(
    [encoder_input_data_train, decoder_input_data_train],
    decoder_output_data_train,
    batch_size=32,
    epochs=15,
    validation_split=0.2,
    callbacks=[checkpoint]
)

Epoch 1/15
80/80 [==============================] - 10s 55ms/step - loss: 3.9090 - accuracy: 0.0122 - val_loss: 3.8718 - val_accuracy: 0.0072
Epoch 2/15
80/80 [==============================] - 2s 29ms/step - loss: 3.6252 - accuracy: 0.0083 - val_loss: 3.6352 - val_accuracy: 0.0126
Epoch 3/15
80/80 [==============================] - 2s 27ms/step - loss: 3.3672 - accuracy: 0.0117 - val_loss: 3.6369 - val_accuracy: 0.0131
Epoch 4/15
80/80 [==============================] - 2s 27ms/step - loss: 3.1887 - accuracy: 0.0164 - val_loss: 3.7072 - val_accuracy: 0.0192
Epoch 5/15
80/80 [==============================] - 2s 27ms/step - loss: 3.0127 - accuracy: 0.0235 - val_loss: 3.8512 - val_accuracy: 0.0253
Epoch 6/15
80/80 [==============================] - 2s 27ms/step - loss: 2.8326 - accuracy: 0.0348 - val_loss: 3.8907 - val_accuracy: 0.0366
Epoch 7/15
80/80 [==============================] - 3s 36ms/step - loss: 2.6533 - accuracy: 0.0501 - val_loss: 3.9499 - val_accuracy: 0.0418
Epoch 8/15
8

In [18]:
from keras.models import load_model

model = load_model('seq2seq_model.h5')

# 훈련된 모델을 사용하여 시퀀스를 디코딩
encoder_model = Model(model.input[0], model.layers[6].output)

decoder_state_input_h = Input(shape=(latent_dim * 2,))
decoder_state_input_c = Input(shape=(latent_dim * 2,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm = model.layers[5]
decoder_dense = model.layers[7]

decoder_outputs, state_h, state_c = decoder_lstm(
    model.input[1], initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [model.input[1]] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

TypeError: call() got an unexpected keyword argument 'initial_state'

In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer_decoder.word_index['<start>']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = tokenizer_decoder.index_word[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == '<end>' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence